In [1]:
import torch
import tiktoken

In [2]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special = {"<|endoftext|>"})

        for i in range(0, len(token_ids)-max_length, stride):
            input_chunck = token_ids[i : i+max_length]
            target_chunck = token_ids[i+1 : i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunck))
            self.target_ids.append(torch.tensor(target_chunck))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [3]:
def create_dataloader_v1(txt, batch_size = 4, max_length = 256,
                         stride = 128, shuffle = True, drop_last = True,
                         num_workers = 0):  # cant increase num_workers due to error, to increase mode the GPTDatasetV1 class to .py file
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size = batch_size,
        shuffle = shuffle,
        drop_last = drop_last,
        num_workers = num_workers
    )
    return dataloader

In [4]:
with open("Datasets/the-verdict.txt", 'r', encoding = "utf-8") as f:
    book_text = f.read()

In [5]:
max_length = 4
dataloader = create_dataloader_v1(
    book_text, batch_size = 8, max_length = max_length, stride = max_length, shuffle = False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [6]:
print("Inputs:\n", inputs)
print("Input shape:\n", inputs.shape,"\n")
print("Targets:\n", targets)
print("Target shape:\n", targets.shape)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input shape:
 torch.Size([8, 4]) 

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
Target shape:
 torch.Size([8, 4])


<h2>Token Embedding layer</h2>

In [8]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
token_embeddings = token_embedding_layer(inputs)

print(token_embeddings.shape)

torch.Size([8, 4, 256])


<h2>Positional Embedding layer</h2>

In [10]:
context_length = max_length

pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_length))

print(pos_embeddings.shape)

torch.Size([4, 256])


<h2>Input Embedding layer</h2>

In [12]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
